## Import the necessary libraries

In [1]:
import numpy as np
from scipy.linalg import lstsq
import torch

In [2]:
A = np.array([
    [1,5,3],
    [2,8,6],
    [4,9,7]
])

b = np.array([
    [10],
    [12],
    [6]
])

In [3]:
x = np.linalg.solve(A,b)
x

array([[-4.],
       [ 4.],
       [-2.]])

In [4]:
# Check Ax = b
# Matrix Product
b == np.matmul(A,x)

array([[ True],
       [ True],
       [ True]])

## Overdetermined System

In [5]:
A = np.array([
    [1,  2,  3],
    [4,  5,  6],
    [7,  8,  9],
    [10, 11, 12],
    [13, 14, 15],
    [16, 17, 18],
    [19, 20, 21]
])

b = np.array([
    [5],
    [7],
    [11],
    [13],
    [17],
    [19],
    [23]
])
b = np.array([5,7,11,13,17,19,23])

In [6]:
# With Numpy
# Using least square
result = np.linalg.lstsq(A,b, rcond=None)
x,residuals,rank,singular_values = result
result

(array([-0.95238095,  0.33333333,  1.61904762]),
 array([], dtype=float64),
 2,
 array([5.75134792e+01, 1.78877318e+00, 3.10802681e-15]))

In [7]:
# With Scipy
result = lstsq(A,b)
x,residuals,rank,singular_values = result
result

(array([-0.95238095,  0.33333333,  1.61904762]),
 array([], dtype=float64),
 2,
 array([5.75134792e+01, 1.78877318e+00, 3.10802681e-15]))

In [8]:
# With PyTorch
a = torch.tensor(A, dtype=torch.float)
b = torch.tensor(b, dtype=torch.float)

result  = torch.linalg.lstsq(a, b, driver='gelsd') #.solution
x,residuals,rank,singular_values = result
result

torch.return_types.linalg_lstsq(
solution=tensor([-0.9524,  0.3333,  1.6190]),
residuals=tensor([]),
rank=tensor(2),
singular_values=tensor([5.7513e+01, 1.7888e+00, 1.4175e-06]))

In [9]:
torch.linalg.pinv(a) @ b

tensor([-0.9524,  0.3333,  1.6190])

In [10]:
x,residuals,rank,singular_values = result
x

tensor([-0.9524,  0.3333,  1.6190])

In [11]:
a.matmul(x)

tensor([ 4.5714,  7.5714, 10.5714, 13.5714, 16.5714, 19.5714, 22.5714])

## Underdetermined System

In [12]:
A = np.array([
    [1,  2,  3],
    [4,  5,  6],
])

b = np.array([
    [5],
    [7],
])

# With PyTorch
a = torch.tensor(A, dtype=torch.float)
b = torch.tensor(b, dtype=torch.float)

result  = torch.linalg.lstsq(a, b, driver='gelsd') #.solution
x,residuals,rank,singular_values = result
result

torch.return_types.linalg_lstsq(
solution=tensor([[-1.6111],
        [ 0.2222],
        [ 2.0556]]),
residuals=tensor([]),
rank=tensor(2),
singular_values=tensor([9.5080, 0.7729]))

In [13]:
# Alternate method
torch.linalg.pinv(a) @ b

tensor([[-1.6111],
        [ 0.2222],
        [ 2.0556]])

In [14]:
a.matmul(x)

tensor([[5.0000],
        [7.0000]])

# Orthogonal Matching Pursuit

In [15]:
A = np.array([
    [1,  2,  3],
    [4,  5,  6],
    [7,  8,  9],
    [10, 11, 12],
    [13, 14, 15],
    [16, 17, 18],
    [19, 20, 21]
])

b = np.array([
    [5],
    [7],
    [11],
    [13],
    [17],
    [19],
    [23]
])
b = np.array([5,7,11,13,17,19,23])
A = torch.tensor(A, dtype=torch.float)
Y = torch.tensor(b, dtype=torch.float)
Y.shape

torch.Size([7])

In [16]:
M = A.shape[0]
N = A.shape[1]

M<N # Underdetermined Equations

False

### Estimate 1st iterations

In [17]:
print(A.transpose(0,1).matmul(Y))
i1 = torch.argmax(A.transpose(0,1).matmul(Y))
print('Max Iterations :',i1.item())

# Augmented Matrix for 1st iterations
A1 = A[:,i1.item()] # 5th columns
print('Augmented Matrix  :',A1)

# Estimate 1st iterations
'''
    Solve least squre 
    min(||y - A1 x1||^2)
    x1= (A1^T A1)^-1 A1^T Y
'''

x1 = ((A1.transpose(-1, 0).matmul(A1))**(-1)) * (A1.transpose(-1, 0)@Y)
print('x1 :',x1)

# Residue after the ist iterations
# r1 = Y-A1x1
r1 = Y - A1.mul(x1)
print('Residue :',r1)

tensor([1202., 1297., 1392.])
Max Iterations : 2
Augmented Matrix  : tensor([ 3.,  6.,  9., 12., 15., 18., 21.])
x1 : tensor(1.1048)
Residue : tensor([ 1.6857,  0.3714,  1.0571, -0.2571,  0.4286, -0.8857, -0.2000])


## 2nd Iterations
Find the projections of r1 i.e residue from the 1st iterations on each column of A

In [18]:
i2 = [i1.item()]
temp =A.transpose(0,1)@r1
print(temp)
if len(temp.shape) == 1:
    i2.append(torch.argmax(temp).item())
else:
    for k in range(temp.shape[0]):
        i2.append(torch.argmax(temp[k]).item())     
print('Max Iterations :',i2)
i2 = list(set(i2))
print('Max Iterations :',i2)

# Augmented Matrix for 1st iterations
A2 = A[:,list(set(i2))] # 2nd and 5the columns
print('Augmented Matrix  :\n',A2)

# Estimate 2nd iterations
'''
    Solve least squre 
    min(||y - A2 x2||^2)
    x2= (A2^T A2)^-1 A2^T y
'''

x2 = torch.inverse(A2.transpose(-1, 0).matmul(A2)) @ (A2.transpose(-1, 0)@Y)
print('x1 :',x2)

# Residue after the ist iterations
# r2 = Y-A2x2
r2 = Y - A2@x2
print('Residue :\n',r2)

tensor([-4.4001e+00, -2.2001e+00, -7.0572e-05])
Max Iterations : [2, 2]
Max Iterations : [2]
Augmented Matrix  :
 tensor([[ 3.],
        [ 6.],
        [ 9.],
        [12.],
        [15.],
        [18.],
        [21.]])
x1 : tensor([1.1048])
Residue :
 tensor([ 1.6857,  0.3714,  1.0571, -0.2571,  0.4286, -0.8857, -0.2000])


In [19]:
r2.to(torch.int)

tensor([1, 0, 1, 0, 0, 0, 0], dtype=torch.int32)

In [20]:
y = torch.tensor([0,2,3,5], dtype=torch.float)
a = torch.tensor([
    [1, 0, 1, 0, 0, 1],
    [0, 1, 1, 1, 0, 0],
    [1, 0, 0, 1, 1, 0],
    [0, 1, 0, 0, 1, 1]
], dtype=torch.float)

y.shape, a.shape

(torch.Size([4]), torch.Size([4, 6]))

## OMP

In [21]:
'''
    A = N X M Matrix
    Y = 1 X N Matrix
    K = Number of iterations

'''
def maxIndex(A,r, i):
    temp =A.transpose(0,1)@r
    #print(temp)
    print(temp.shape)
    if len(temp.shape) == 1:
        i.append(torch.argmax(temp).item())
    else:
        for k in range(temp.shape[0]):
            i.append(torch.argmax(temp[k]).item())              
    return list(set(i))
def iterations(A_bar,y):
    temp = A_bar.transpose(-1, 0).matmul(A_bar)
    #print(temp)
    #print(len(temp.shape))
    if len(temp.shape) == 0:
        x_bar = (temp)**(-1) * (A_bar.transpose(-1, 0)@y)
        x_bar
    else:
        x_bar = torch.inverse(temp) @ (A_bar.transpose(-1, 0)@y)
        x_bar
    return x_bar

def OMP(A,Y,K):
    r = Y
    x = [0 for i in range(A.shape[1])]
    k = 0
    Index = []
    while k <= K:                 #torch.argmax(r).item()>0.01:
        # Max Index
        Index = maxIndex(A,r,Index)

        # Augmented Matrix
        A_bar =A [:,Index]

        # Least Square value
        X_bar = iterations(A_bar,Y)

        # Residue
        r = Y - A_bar @ X_bar
        print(Index)
        #print(X_bar)
        print(r)
        k +=1
        
    for i in range(len(X_bar)):
        x[Index[i]] = X_bar[i].item()
    return x 

print(A.shape)
print(Y.shape)
OMP(A,Y, K = 5)   

torch.Size([7, 3])
torch.Size([7])
torch.Size([3])
[2]
tensor([ 1.6857,  0.3714,  1.0571, -0.2571,  0.4286, -0.8857, -0.2000])
torch.Size([3])
[2]
tensor([ 1.6857,  0.3714,  1.0571, -0.2571,  0.4286, -0.8857, -0.2000])
torch.Size([3])
[2]
tensor([ 1.6857,  0.3714,  1.0571, -0.2571,  0.4286, -0.8857, -0.2000])
torch.Size([3])
[2]
tensor([ 1.6857,  0.3714,  1.0571, -0.2571,  0.4286, -0.8857, -0.2000])
torch.Size([3])
[2]
tensor([ 1.6857,  0.3714,  1.0571, -0.2571,  0.4286, -0.8857, -0.2000])
torch.Size([3])
[2]
tensor([ 1.6857,  0.3714,  1.0571, -0.2571,  0.4286, -0.8857, -0.2000])


[0, 0, 1.1047619581222534]

In [22]:


i =[]
print(a.shape)
print(y.shape)
maxIndex(a,y, i)

torch.Size([4, 6])
torch.Size([4])
torch.Size([6])


[4]

In [23]:
print(A.shape)
print(Y.shape)
i =[]
maxIndex(A, Y, i)

torch.Size([7, 3])
torch.Size([7])
torch.Size([3])


[2]

In [24]:
print(a.shape)
print(y.shape)
OMP(a,y, 5)

torch.Size([4, 6])
torch.Size([4])
torch.Size([6])
[4]
tensor([ 0.,  2., -1.,  1.])
torch.Size([6])
[1, 4]
tensor([ 0.0000e+00, -2.3842e-07,  0.0000e+00,  0.0000e+00])
torch.Size([6])
[0, 1, 4]
tensor([0., 0., 0., 0.])
torch.Size([6])
[0, 1, 4]
tensor([0., 0., 0., 0.])
torch.Size([6])
[0, 1, 4]
tensor([0., 0., 0., 0.])
torch.Size([6])
[0, 1, 4]
tensor([0., 0., 0., 0.])


[0.0, 2.0, 0, 0, 3.0, 0]